# Skill estimation using Stan

In [10]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

## Model that defined skill's distribution with more features

In [4]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
  int DA[E];                  # number of days for PA plays the game
  int DB[E];                  # number of days for PB plays the game
  int RA[E];                  # race for PA plays the game
  int RB[E];                  # race for PB plays the game  
  
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [4]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
  int RA[E];                  # race for PA plays the game
  int RB[E];                  # race for PB plays the game  
  
}
parameters {
  vector [N] skillA;           # skill values for each player
  vector [N] skillB;           # skill values for each player
  vector [N] skillC;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [154]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  int<lower=0,upper=2> RA[E];                  // player info between each game
  int<lower=0,upper=2> RB[E];                  // 
  
}
parameters {
  vector [N] skillP;           // skill values for each player for different races
  vector [N] skillT;
  vector [N] skillZ;
}

model{
  for (i in 1:N){ 
    skillP[i]~normal(0,3); 
    skillT[i]~normal(0,3); 
    skillZ[i]~normal(0,3); 
  }
  for (i in 1:E){
    if (RA[i] == 0 && RB[i] == 0){win[i] ~ bernoulli_logit( (scale)*(skillP[PA[i]]-skillP[PB[i]]) );}
    if (RA[i] == 0 && RB[i] == 1){win[i] ~ bernoulli_logit( (scale)*(skillP[PA[i]]-skillT[PB[i]]) );}
    if (RA[i] == 0 && RB[i] == 2){win[i] ~ bernoulli_logit( (scale)*(skillP[PA[i]]-skillZ[PB[i]]) );}
    if (RA[i] == 1 && RB[i] == 0){win[i] ~ bernoulli_logit( (scale)*(skillT[PA[i]]-skillP[PB[i]]) );}
    if (RA[i] == 1 && RB[i] == 1){win[i] ~ bernoulli_logit( (scale)*(skillT[PA[i]]-skillT[PB[i]]) );}
    if (RA[i] == 1 && RB[i] == 2){win[i] ~ bernoulli_logit( (scale)*(skillT[PA[i]]-skillZ[PB[i]]) );}
    if (RA[i] == 2 && RB[i] == 0){win[i] ~ bernoulli_logit( (scale)*(skillZ[PA[i]]-skillP[PB[i]]) );}
    if (RA[i] == 2 && RB[i] == 1){win[i] ~ bernoulli_logit( (scale)*(skillZ[PA[i]]-skillT[PB[i]]) );}
    if (RA[i] == 2 && RB[i] == 2){win[i] ~ bernoulli_logit( (scale)*(skillZ[PA[i]]-skillZ[PB[i]]) );}
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [155]:
sm = pystan.StanModel(model_code = skill_model)

In [151]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_1.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_1.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [209]:
def load_data(dir='data/', pKeep=1.0, nEdge=9, nKeep=15, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    raceA, raceB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    race = {'P':0, 'T':1, 'Z':2}
    for i in np.random.permutation(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 
                raceA.append(race[csv[6]])
                raceB.append(race[csv[7]])

    return nplayers,playerA,playerB,raceA, raceB, wins

In [210]:
nplayers,playerA,playerB,raceA, raceB, wins = load_data()

In [211]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])
print('raceA', raceA[:10])
print('raceB', raceB[:10])

summary: 
# players 999
# games 7385
player A [611, 364, 572, 31, 471, 341, 426, 114, 330, 510]
player B [140, 137, 313, 405, 583, 807, 510, 72, 547, 635]
wins [0, 1, 0, 1, 1, 0, 0, 1, 1, 1]
raceA [1, 2, 2, 0, 2, 0, 0, 0, 1, 2]
raceB [0, 0, 1, 1, 2, 0, 2, 0, 1, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [212]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.3,
    'win':wins,
    'PA': playerA,
    'PB': playerB,
    'RA': raceA,
    'RB': raceB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [ ]:
fit = sm.sampling(data=skill_data, iter=10000, chains=4)

In [ ]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [ ]:
samples

In [ ]:
samples['skillP'].shape # 2*100 iterations? 999 players

In [ ]:
samples['skillP'].mean(0)

Remember to save the prediction!

In [ ]:
with open('skill_hat_PTZ.pkl', 'wb') as f: 
    pickle.dump(samples, f)

## Sample Model Evaluation

In [ ]:
skill_hat = pickle.load(open('skill_hat_PTZ.pkl', 'rb'))

In [ ]:
def load_valid_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player  # id to name lookup


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    race = {'P':0, 'T':1, 'Z':2}
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        nplays[a,b] += 1
        nplays[b,a]+=1
        nwins[a,b] += aw
        nwins[b,a] += bw
        games.append((a,b,aw,race[csv[6]],race[csv[7]]))
    
    return nplayers, nplays, nwins, games

In [ ]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data()

In [ ]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

In [ ]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skills, games):
    
    loss = 0.
    race = {0: 'skillP', 1: 'skillT', 2:'skillZ'}
    for game in games:
        a,b,aw,ra,rb = game
        prob_hat = logit( 0.1*(skills[race[ra]][:,a]-skills[race[rb]][:,b]) ).mean()
        loss += np.logical_xor(prob_hat >= 0.5, aw)
    
    loss /= len(games)
    
    return loss


In [ ]:
loss = prediction_loss(skill_hat, games_val)

In [ ]:
loss